In [6]:
import pandas as pd
import numpy as np
import re
import pickle
import time, os

pd.set_option('display.max_rows', 308)

In [7]:
#compile pdfs into dataframe
columns = ['Circuit', 'Tiers', 'Start Date', 'Key Communities', 'End Date']
data = pd.DataFrame(columns = columns)

data = pd.concat([
    pd.read_csv("PSPS - 06.21.19.csv", names = columns),
    pd.read_csv("PSPS - 10.05.19.csv", names = columns),
    pd.read_csv("PSPS - 10.09.19.csv", names = columns),
    pd.read_csv("PSPS - 10.10.19.csv", names = columns),
    pd.read_csv("PSPS - 10.23.19.csv", names = columns),
    pd.read_csv("PSPS - 10.26.19.csv", names = columns),
    pd.read_csv("PSPS - 10.31.18.csv", names = columns),
    pd.read_csv("PSPS - 11.20.19.csv", names = columns)
], ignore_index = True, sort = False)

## Data Cleaning

In [11]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql


In [161]:
#drop columns
data = data.drop(columns = ['Circuit', 'Tiers'])

#limit key communities to one
data = data.applymap(lambda x: x.split('\r', 1)[0])
data = data.applymap(lambda x: x.split(',', 1)[0])
#data['Circuit'] = data['Circuit'].apply(lambda x: re.sub(r'[^a-zA-Z]', '', x))

#remove transmission line data
transmission_entries = data[data['Key Communities'] == 'Transmission Line'].index
data.drop(transmission_entries, inplace=True)

#remove rows with invalid dates
data['Start Date'] = data['Start Date'].replace(r'^([^0-9]*)$', np.nan, regex=True)
data = data.dropna()

#remove timestamps
data['Start Date'] = data['Start Date'].apply(lambda x: x.split(' ', 1)[0])
data['End Date'] = data['End Date'].apply(lambda x: x.split(' ', 1)[0])

#remove rows without full dates
data = data[data['Start Date'].str.contains('/')]

#format date column
data['Start Date'] = data['Start Date'].astype('datetime64')
data['End Date'] = data['End Date'].astype('datetime64') 

#proper capitalization on key communities
data['Key Communities'] = data['Key Communities'].apply(lambda x: x.title())

# SQL

In [222]:
# CREATE TABLE locations (
# Index int NOT NULL,
# "Key Communities" text NOT NULL,
# "Latitude" float NOT NULL,
# "Longitude" float NOT NULL
# );

connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'project3',    # DB that we are connecting to
    'port': 5432          # port for psql
}

connection = pg.connect(**connection_args)

In [ ]:
query = "SELECT * FROM locations;"

locations = pd_sql.read_sql(query, connection)
locations = locations.drop(columns = 'index')

In [162]:
data = data.merge(locations, on = 'Key Communities')

In [163]:
data['Shutoff']=True

columns = ['Key Communities', 'Start Date', 'Shutoff', 'Latitude', 'Longitude', 'End Date']
data = data[columns]

In [209]:
pickle.dump(data,open('data','wb'))

In [12]:
data = pickle.load(open('data','rb'))

## non-target data

In [130]:
comm_date_set = set()

def entered_data(row):
    index = row.name
    comm = row['Key Communities']
    date = row['Start Date']
    
    comm_date_set.add((comm, date))
    
data.apply(helper, axis = 1);

In [204]:
pairings = set()

for city in data['Key Communities'].unique():
    for days in pd.date_range(end = '2019-11-30', periods = 91):
        pairings.add((city, days))

pairings -= comm_date_set

In [215]:
power_on = []
for pair in pairings:
    power_ = {}
    power_['Key Communities'] = pair[0]
    power_['Start Date'] = pair[1]
    power_on.append(power_)
add_data = pd.DataFrame(power_on)
add_data['Shutoff'] = False

In [223]:
locations = pickle.load(open('locations','rb'))

In [230]:
add_data = add_data.merge(locations, on = 'Key Communities')

In [233]:
add_data['End Date'] = None

In [236]:
pickle.dump(add_data,open('add_data','wb'))

In [260]:
add_data = pickle.load(open('add_data','rb'))

In [262]:
total_data = pd.concat([data, add_data], ignore_index = True, sort = False)

## scrape weather data

In [197]:
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
from IPython.core.display import display, HTML
import json

In [198]:
ua = UserAgent()
user_agent = {'User-agent': ua.random}
print(user_agent)

{'User-agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.17 Safari/537.36'}


In [199]:
def weather(row):
    index = row.name
    
    darksky_key = <PUT KEY HERE>
    TIMESTAMP = str(row['Start Date'].timestamp()).split('.',1)[0]
    LAT = str(row['Latitude'])
    LONG = str(row['Longitude'])
    darksky_url = 'https://api.darksky.net/forecast/' + darksky_key + '/' + LAT + ',' + LONG + ',' + TIMESTAMP + '?exclude=currently,flags'
    
    response = requests.get(darksky_url, headers = user_agent)
    weather_data = response.json()

    total_data.loc[index, 'High'] = weather_data['daily']['data'][0]['temperatureHigh']
    total_data.loc[index, 'Low'] = weather_data['daily']['data'][0]['temperatureLow']
    total_data.loc[index, 'Humidity'] = weather_data['daily']['data'][0]['humidity']
    total_data.loc[index, 'Dewpoint'] = weather_data['daily']['data'][0]['dewPoint']
    total_data.loc[index, 'Pressure'] = weather_data['daily']['data'][0]['pressure']
    total_data.loc[index, 'Windspeed'] = weather_data['daily']['data'][0]['windSpeed']
    total_data.loc[index, 'Windgust'] = weather_data['daily']['data'][0]['windGust']
    total_data.loc[index, 'Cover'] = weather_data['daily']['data'][0]['cloudCover']

In [322]:
total_data.apply(weather, axis = 1);

In [327]:
pickle.dump(total_data,open('total_data','wb'))

In [328]:
total_data = pickle.load(open('total_data','rb'))

## FIX MY DATA


In [70]:
total_data = pickle.load(open('total_data','rb'))

In [149]:
weather_by_day = total_data.loc[:, ['Key Communities', 'Start Date', 'High', 'Low', 'Humidity', 'Dewpoint', 'Pressure',
       'Windspeed', 'Windgust', 'Cover']].copy(deep=True)

target_df = total_data.loc[total_data['Shutoff']==True, ['Key Communities', 'Start Date', 'Shutoff', 'Latitude', 'Longitude',
       'End Date']].copy(deep=True)

other_data = total_data.loc[total_data['Shutoff']==False, ['Key Communities', 'Start Date', 'Shutoff', 'Latitude', 'Longitude',
       ]].copy(deep=True)

In [152]:
target_df['Day'] = target_df.apply(lambda row: pd.date_range(row['Start Date'], row['End Date']), axis=1)

In [153]:
target_df = target_df.explode('Day').reset_index() \
        .drop(columns=['index', 'Start Date'])

In [154]:
other_data = other_data.rename(columns={'Start Date': 'Day'})

In [155]:
target_and_other = pd.merge(target_df,other_data, on=['Key Communities', 'Day', 'Latitude', 'Longitude'], how='outer')

In [156]:
#it worked!! removed false shutoff rows that overlapped with target data
print([target_df.shape, other_data.shape, target_and_other.shape])

[(4643, 6), (27468, 5), (30811, 7)]


In [157]:
add_to_target_df = target_and_other[target_and_other['Shutoff_x'].isnull()]
add_to_target_df = add_to_target_df.drop(columns = ['Shutoff_x']) \
                    .rename(columns = {'Shutoff_y':'Shutoff'})

In [264]:
df_final = pd.concat([target_df, add_to_target_df], sort=False).drop(columns=['End Date'])

In [173]:
weather_by_day = weather_by_day.rename(columns = {'Start Date':'Day'})

In [201]:
missing_weather = df_final[df_final['Day']=='2019-06-09T00:00:00.000000000'] \
                    .drop_duplicates() \
                    .drop(columns = 'Shutoff') \
                    .rename(columns = {'Day': 'Start Date'})

In [208]:
def weather2(row):
    index = row.name
    
    darksky_key = <PUT KEY HERE>
    TIMESTAMP = str(row['Start Date'].timestamp()).split('.',1)[0]
    LAT = str(row['Latitude'])
    LONG = str(row['Longitude'])
    darksky_url = 'https://api.darksky.net/forecast/' + darksky_key + '/' + LAT + ',' + LONG + ',' + TIMESTAMP + '?exclude=currently,flags'
    
    response = requests.get(darksky_url, headers = user_agent)
    weather_data = response.json()

    missing_weather.loc[index, 'High'] = weather_data['daily']['data'][0]['temperatureHigh']
    missing_weather.loc[index, 'Low'] = weather_data['daily']['data'][0]['temperatureLow']
    missing_weather.loc[index, 'Humidity'] = weather_data['daily']['data'][0]['humidity']
    missing_weather.loc[index, 'Dewpoint'] = weather_data['daily']['data'][0]['dewPoint']
    missing_weather.loc[index, 'Pressure'] = weather_data['daily']['data'][0]['pressure']
    missing_weather.loc[index, 'Windspeed'] = weather_data['daily']['data'][0]['windSpeed']
    missing_weather.loc[index, 'Windgust'] = weather_data['daily']['data'][0]['windGust']
    missing_weather.loc[index, 'Cover'] = weather_data['daily']['data'][0]['cloudCover']

In [211]:
#adds community+day combos to weather_data
missing_weather.apply(weather2, axis = 1);

In [215]:
missing_weather = missing_weather.rename(columns = {'Start Date':'Day'}) \
                    .drop(columns = ['Latitude', 'Longitude'])
missing_weather

,Key Communities,Day,High,Low,Humidity,Dewpoint,Pressure,Windspeed,Windgust,Cover
1,Bangor,2019-06-09,82.79,66.98,0.23,32.03,1016.4,7.22,25.28,0.12
19,Chico,2019-06-09,88.69,63.99,0.21,28.17,1017.6,9.15,19.56,0.11
76,Oroville,2019-06-09,88.58,65.70,0.21,29.63,1016.8,9.08,19.93,0.11
201,Magalia,2019-06-09,77.57,65.96,0.21,24.30,1018.0,8.76,29.20,0.12
228,Butte Meadows,2019-06-09,69.68,52.96,0.27,20.13,1019.3,9.28,26.44,0.13
232,Paradise,2019-06-09,83.09,68.19,0.20,25.49,1017.7,6.03,21.50,0.11
321,Berry Creek,2019-06-09,78.33,62.85,0.24,26.72,1017.4,8.80,26.33,0.12


In [218]:
#weather_by_day.shape = (28929, 10)
weather_by_day = pd.concat([weather_by_day, missing_weather], sort=False)
#weather_by_day.shape = (28936, 10)

In [332]:
weather_by_day = weather_by_day.drop_duplicates(['Key Communities','Day'],keep= 'last')
weather_by_day.shape

(28044, 10)

In [338]:
model_data = pd.merge(df_final, weather_by_day, on = ['Key Communities', 'Day'])

In [342]:
model_data['Shutoff'] = model_data['Shutoff'].astype(bool)

In [343]:
pickle.dump(model_data,open('model_data','wb'))